In [ ]:

import os
import math
import random
import subprocess
from math import ceil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Reprodutibilidade
random.seed(42)
np.random.seed(42)

# ================== CONFIGURAÇÃO ==================
REGION_NAME = "sul"
CONTROL_SHEET_PATH = rf"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa\tables\control_{REGION_NAME}.xlsx"

OBSERVED_DATA_PATH = r"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa\dados\CURVE_DATABASE_Severity&Defol.xlsx"
OBSERVED_SHEET_NAME = "organizados"

PARAMS_FILE = r"C:\DSSAT48\Soybean\disease_parameters.txt"
DSSAT_EXE   = r"C:\dssat-csm-os-develop\build\bin\Debug\dscsm048.exe"
EXP_DIR     = r"C:\DSSAT48\Soybean"
DSSAT_OUTPUT_FILE = os.path.join(EXP_DIR, "DISEASE_DEVELOPMENT.OUT")

# >>> Diretório fixo de saída no Desktop <<<
OUTPUT_DIR = Path(r"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# NDS/DAE_START fixos nesta etapa regional
FIXED_NDS       = 50000
FIXED_DAE_START = 30

# Tamanho das fases
N_GLOBAL = 220     # Fase A (exploração)
N_LOCAL  = 480     # Fase B (refino)

# Paciência para encolher a trust region
PATIENCE = 60
SHRINK   = 0.65    # encurtar desvio em 35% quando não houver melhora

# ================== SENSIBILIDADE (μ* do Morris) ==================
MU_STAR = {
    'LDMin':        1045.387732,
    'LESLIFEMAX':   1003.152347,
    'TOT_D':         882.853503,
    'RVERHULST':     514.341934,
    'TMAX_D':        482.307563,
    'LESIONAGEOPT':  412.321551,
    'LESION_S':      399.491876,
    'YMAX':          322.785414,
    'KVERHULST':     208.737913,
    'TMIN_D':         79.160729,
    'COF_B':          73.936855,
    'TOT_G':          71.956077,
    'COF_A':          49.048563,
    'TMAX_G':         31.667266,
    'TMIN_G':          2.183453,
}

# ================== FAIXAS DE BUSCA ==================
def get_param_ranges():
    return {
        'LESION_S':     (1.75e-6, 2.50e-6),
        'KVERHULST':    (400000, 571400),  # inteiro  << corrigido
        'RVERHULST':    (0.5, 4.0),
        'YMAX':         (0.20, 0.40),
        'COF_A':        (0.10, 0.30),
        'COF_B':        (1.00, 3.00),
        'TMIN_G':       (8.0, 11.0),
        'TOT_G':        (20.0, 25.0),
        'TMAX_G':       (28.5, 33.0),
        'TMIN_D':       (8.0, 12.0),
        'TOT_D':        (21.0, 25.0),
        'TMAX_D':       (30.0, 34.0),
        'LDMin':        (5.0, 16.0),
        'LESIONAGEOPT': (0.1, 0.9),
        'LESLIFEMAX':   (20.0, 70.0),
    }

def nominal_from_ranges(ranges):
    """Valor 'central' nominal por parâmetro."""
    nom = {}
    for k, (a, b) in ranges.items():
        nom[k] = (a + b) / 2.0
    nom['KVERHULST'] = int(round(nom['KVERHULST']))
    nom['NDS']       = FIXED_NDS
    nom['DAE_START'] = FIXED_DAE_START
    return nom

def weights_from_mu_star(mu):
    """Normaliza μ* para [0.15, 1.0] (evita peso zero) e devolve dict."""
    vals = np.array(list(mu.values()), dtype=float)
    if np.allclose(vals.std(), 0.0):
        return {k: 1.0 for k in mu}
    vmin, vmax = float(vals.min()), float(vals.max())
    w = {}
    for k, v in mu.items():
        ww = (v - vmin) / (vmax - vmin) if vmax > vmin else 1.0
        w[k] = 0.15 + 0.85 * ww
    return w

WEIGHTS = weights_from_mu_star(MU_STAR)
RANGES  = get_param_ranges()
NOMINAL = nominal_from_ranges(RANGES)

# ================== I/O DSSAT ==================
def write_params_to_file(params, header_lines, disease_name, sim_number=1):
    """Escreve um conjunto de parâmetros no arquivo do DSSAT."""
    new_id = f"D{sim_number:05d}"
    line = (
        f"{new_id:<7s} {disease_name:<17s}"
        f"{int(params['NDS']):>10d}{params['LESION_S']:>16.6f}"
        f"{int(params['KVERHULST']):>16d}{params['RVERHULST']:>16.3f}"
        f"{params['YMAX']:>16.3f}{params['COF_A']:>16.3f}{params['COF_B']:>16.3f}"
        f"{params['TMIN_G']:>16.3f}{params['TOT_G']:>16.3f}{params['TMAX_G']:>16.3f}"
        f"{params['TMIN_D']:>16.3f}{params['TOT_D']:>16.3f}{params['TMAX_D']:>16.3f}"
        f"{params['LDMin']:>16.3f}{params['LESIONAGEOPT']:>16.3f}{params['LESLIFEMAX']:>16.3f}"
        f"{int(params['DAE_START']):>16d}"
    )
    with open(PARAMS_FILE, 'w', encoding='utf-8') as f:
        f.writelines(header_lines)
        f.write(line + '\n')

def run_dssat_simulation(experiment_name):
    try:
        subprocess.run(
            [DSSAT_EXE, "A", experiment_name],
            cwd=EXP_DIR, check=True, capture_output=True, text=True, encoding='latin1'
        )
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# ================== LEITURA ROBUSTA DO OUT ==================
def read_disease_out_last_run(path_out: str) -> pd.DataFrame:
    """
    Lê DISEASE_DEVELOPMENT.OUT procurando o cabeçalho real, e retorna
    o último RUN com colunas padronizadas:
    ['DAS','LAI_HEALTH','LA_INFECT','LA_DISEASE','LAI_TOTAL']
    """
    with open(path_out, 'r', encoding='latin1', errors='ignore') as f:
        lines = f.readlines()

    header_idx = None
    target_keys = ["RUN", "YYDOY", "DAS", "LAI_HEALTH", "LA_DISEASE", "LAI_TOTAL"]
    for idx, ln in enumerate(lines):
        if all(k in ln for k in target_keys):
            header_idx = idx
            break
    if header_idx is None:
        raise ValueError("Cabeçalho não encontrado em DISEASE_DEVELOPMENT.OUT")

    colnames = [
        "RUN","YYDOY","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","NEW_LOSS",
        "LWD","RH","FT","LAI_TOTAL","SUM7","NSPRAYS","FUNG_ACT"
    ]
    df = pd.read_fwf(path_out, skiprows=header_idx+1, names=colnames, engine="python", encoding='latin1')

    # tipos
    num_cols = ["RUN","YYDOY","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","LAI_TOTAL","SUM7","NSPRAYS"]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna(subset=["RUN", "DAS"]).copy()
    if df.empty:
        raise ValueError("Sem linhas numéricas após o cabeçalho no OUT.")

    last_run = int(df["RUN"].max())
    df = df[df["RUN"] == last_run].copy()

    df["DAS"] = df["DAS"].astype(int)
    df = df.sort_values("DAS")

    keep = ["DAS","LAI_HEALTH","LAI_TOTAL","LA_DISEASE","LA_INFECT"]
    for k in keep:
        if k not in df.columns:
            df[k] = np.nan
    df = df[keep]
    df = df.dropna(subset=["DAS"])
    return df

# ================== SEVERIDADE (MONÓTONA) ==================
def calculate_severity(df_last_run: pd.DataFrame, last_obs_day: int = None) -> pd.DataFrame:
    """
    Severidade cumulativa e monótona:
      Severity(%) = 100 * LA_DISEASE / LAI_PEAK
    onde LAI_PEAK é o pico de LAI_TOTAL até last_obs_day (se fornecido),
    caso contrário o pico no período disponível.
    """
    df = df_last_run.copy()
    for c in ["DAS","LAI_TOTAL","LA_DISEASE"]:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    df = df.dropna(subset=["DAS","LAI_TOTAL","LA_DISEASE"]).sort_values("DAS")
    if df.empty:
        return pd.DataFrame(columns=["DAS","Severity(%)"])

    df["DAS"] = df["DAS"].astype(int)

    if last_obs_day is not None and np.isfinite(last_obs_day):
        lai_peak = float(df.loc[df["DAS"] <= int(last_obs_day), "LAI_TOTAL"].max())
        if not np.isfinite(lai_peak) or lai_peak <= 0:
            lai_peak = float(df["LAI_TOTAL"].max())
    else:
        lai_peak = float(df["LAI_TOTAL"].max())

    lai_peak = max(lai_peak, 1e-9)

    sev = 100.0 * df["LA_DISEASE"].clip(lower=0.0) / lai_peak
    sev = np.clip(sev.values, 0.0, 100.0)
    sev = np.maximum.accumulate(sev)  # garante monotonia

    return pd.DataFrame({"DAS": df["DAS"].values, "Severity(%)": sev})

# ================== AVALIAÇÃO ==================
def evaluate_simulation(sim_df, obs_df, curve_id):
    """
    Retorna: rmse, r2, merged_df, sim_df_filtrado, sse, n
    """
    obs_curve_df = obs_df[obs_df['Curva'] == curve_id].copy()
    if obs_curve_df.empty:
        return np.inf, -np.inf, None, None, 0.0, 0

    obs_curve_df['Av_DAS'] = pd.to_numeric(obs_curve_df['Av_DAS'], errors='coerce')
    obs_curve_df['Severidade (%)'] = pd.to_numeric(obs_curve_df['Severidade (%)'], errors='coerce')
    obs_curve_df.dropna(subset=['Av_DAS', 'Severidade (%)'], inplace=True)
    if len(obs_curve_df) < 3:
        return np.inf, -np.inf, None, None, 0.0, 0

    ultimo = int(obs_curve_df['Av_DAS'].max())
    sim_df_filtrado = sim_df[sim_df['DAS'] <= ultimo].copy()

    obs_points = obs_curve_df[['Av_DAS', 'Severidade (%)']].rename(columns={'Av_DAS': 'DAS', 'Severidade (%)': 'obs'})
    merged_df = pd.merge(sim_df_filtrado, obs_points, on='DAS', how='inner')
    if len(merged_df) < 3:
        return np.inf, -np.inf, None, None, 0.0, 0

    resid = merged_df['obs'].values - merged_df['Severity(%)'].values
    sse = float(np.sum(resid**2))
    n = len(merged_df)

    rmse = float(np.sqrt(sse / n))
    r2 = r2_score(merged_df['obs'], merged_df['Severity(%)'])

    return rmse, r2, merged_df, sim_df_filtrado, sse, n

# ================== PLOT ==================
def plot_regional_results(best_params_results, control_df, region_name, outdir: Path):
    n_plots = len(best_params_results)
    n_cols = 3
    n_rows = ceil(n_plots / n_cols) if n_plots > 0 else 1
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5.5, n_rows * 5), constrained_layout=True)
    axes = np.atleast_1d(axes).flatten()

    for i, (curve_id, data) in enumerate(best_params_results.items()):
        ax = axes[i]
        location_info = control_df[control_df['Id_curva'] == curve_id].iloc[0]
        ax.set_title(f"{location_info['LocationName']} ({pd.to_datetime(location_info['PlantingDate']).strftime('%d/%m/%Y')})", fontsize=16)
        ax.scatter(data['merged_df']['DAS'], data['merged_df']['obs'], label='Observado',
                   color='crimson', s=60, zorder=5, edgecolors='black')
        ax.plot(data['sim_df']['DAS'], data['sim_df']['Severity(%)'], label='Simulado', color='black')
        stats_text = f"RMSE = {data['rmse']:.2f}\n$R^2$ = {data['r2']:.2f}"
        ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=12, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        ax.set_ylabel("Severidade (%)", fontsize=14)
        ax.set_xlabel("Dias Após Semeadura (DAS)", fontsize=14)
        ax.tick_params(axis='both', which='major', labelsize=12)
        ax.legend(loc='lower right', fontsize=12)
        ax.set_ylim(0, 100)

    # Desliga eixos sobrando
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle(f'Calibração Biológica Regional - {region_name}', fontsize=22, weight='bold')
    plot_path = outdir / f"grafico_calibracao_biologica_{region_name}.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: '{plot_path}'")
    plt.show()

# ================== SALVAR XLSX COM FALLBACK ==================
def save_params_safely(params_dict: dict, region_name: str, outdir: Path) -> Path:
    df = pd.DataFrame([params_dict])
    xlsx_path = outdir / f"parametros_biologicos_{region_name}.xlsx"
    try:
        with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
            df.to_excel(writer, index=False)
        print(f"Parâmetros salvos em: '{xlsx_path}'")
        return xlsx_path
    except Exception as e:
        csv_path = xlsx_path.with_suffix(".csv")
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"[Aviso] Falha ao salvar XLSX ({e}). Salvo como CSV: '{csv_path}'")
        return csv_path

# ================== AMOSTRAGEM GUIADA POR SENSIBILIDADE ==================
def enforce_constraints(p):
    """Limita aos bounds e garante TMIN < TOT < TMAX; K inteiro; injeta fixos."""
    # bounds
    for k, (a, b) in RANGES.items():
        if k in p:
            p[k] = float(np.clip(p[k], a, b))
    # ordenação T
    for trio in [('TMIN_G','TOT_G','TMAX_G'), ('TMIN_D','TOT_D','TMAX_D')]:
        lo, mid, hi = trio
        v = sorted([p[lo], p[mid], p[hi]])
        p[lo], p[mid], p[hi] = v[0], v[1], v[2]
    # inteiro
    p['KVERHULST'] = int(round(p['KVERHULST']))
    # fixos
    p['NDS']       = FIXED_NDS
    p['DAE_START'] = FIXED_DAE_START
    return p

def sample_global():
    """Amostra global enviesada por μ*: mistura (w * uniforme + (1-w) * nominal)."""
    p = {}
    for k, (a, b) in RANGES.items():
        w = WEIGHTS.get(k, 0.5)  # mais sensível -> mais uniforme
        u = a + (b - a) * random.random()
        x = w * u + (1.0 - w) * NOMINAL[k]
        p[k] = x
    return enforce_constraints(p)

def sample_local(best, base_scale=0.30):
    """
    Perturbação gaussiana ao redor do 'best'.
    sigma_k = base_scale * (b-a) * WEIGHTS[k]
    """
    p = {}
    for k, (a, b) in RANGES.items():
        width = (b - a)
        sigma = base_scale * width * WEIGHTS.get(k, 0.5)
        # baixa probabilidade de mexer em parâmetros pouco sensíveis
        mutate_prob = 0.20 + 0.80 * WEIGHTS.get(k, 0.5)
        if random.random() < mutate_prob:
            val = np.random.normal(loc=best[k], scale=max(sigma, 1e-9))
        else:
            val = best[k]
        p[k] = val
    return enforce_constraints(p)

# ================== OBJETIVO ==================
def objective(params, header, disease_name, control_df, observed_df, run_id=1):
    """Escreve params, roda todos os experimentos e devolve (rmse_regional, SSE_total, N_total, n_ok)."""
    write_params_to_file(params, header, disease_name, sim_number=run_id)

    SSE_total = 0.0
    N_total   = 0
    n_ok      = 0

    for _, ex in control_df.iterrows():
        exp_file_name = f"{ex['Experiment']}.SBX"
        curve_id = ex['Id_curva']

        if not run_dssat_simulation(exp_file_name):
            continue

        if os.path.exists(DSSAT_OUTPUT_FILE):
            try:
                df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)

                # >>> essencial: usa último dia observado p/ LAI_PEAK da curva
                obs_curve = observed_df[observed_df['Curva'] == curve_id]
                if obs_curve.empty:
                    last_obs_day = None
                else:
                    last_obs_day = int(pd.to_numeric(obs_curve['Av_DAS'], errors='coerce').max())

                sim_severity_df = calculate_severity(df_out, last_obs_day=last_obs_day)

                rmse, _, _, _, sse, n = evaluate_simulation(sim_severity_df, observed_df, curve_id)
                if n >= 3 and np.isfinite(rmse):
                    SSE_total += sse
                    N_total   += n
                    n_ok      += 1
            except Exception:
                continue

    if N_total == 0 or n_ok == 0:
        return np.inf, 0.0, 0, 0

    rmse_region = math.sqrt(SSE_total / N_total)
    return rmse_region, SSE_total, N_total, n_ok

# ================== LOOP PRINCIPAL ==================
def main():
    # Carrega controle/observado/header
    try:
        control_df  = pd.read_excel(CONTROL_SHEET_PATH)
        observed_df = pd.read_excel(OBSERVED_DATA_PATH, sheet_name=OBSERVED_SHEET_NAME)
        with open(PARAMS_FILE, 'r', encoding='utf-8') as f:
            all_lines = f.readlines()
            var_index = next(i for i, l in enumerate(all_lines) if l.strip().startswith('@VAR#'))
            header = all_lines[:var_index + 3]
            disease_name = all_lines[var_index + 3].split()[1]
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado - {e.filename}")
        return
    except Exception as e:
        print(f"Ocorreu um erro ao carregar os arquivos: {e}")
        return

    # ===== Fase A: Exploração global =====
    print(f"Fase A (global): {N_GLOBAL} amostras enviesadas por sensibilidade...")
    best_params = None
    best_rmse   = np.inf
    best_meta   = (0, 0, 0)  # (SSE, N_total, n_ok)
    run_id      = 0

    for i in range(1, N_GLOBAL + 1):
        run_id += 1
        cand = sample_global()
        rmse_reg, sse, n_tot, n_ok = objective(cand, header, disease_name, control_df, observed_df, run_id=run_id)
        if np.isfinite(rmse_reg) and rmse_reg < best_rmse:
            best_rmse   = rmse_reg
            best_params = cand.copy()
            best_meta   = (sse, n_tot, n_ok)
            print(f"  [A {i:03d}/{N_GLOBAL}] Melhor RMSE regional = {best_rmse:.4f}  (pontos: {n_tot}, exps ok: {n_ok}/{len(control_df)})")

    if best_params is None:
        print("Nenhuma solução válida na Fase A.")
        return

    # ===== Fase B: Refino local =====
    print(f"\nFase B (local): {N_LOCAL} perturbações gaussianas ∝ μ* ...")
    base_scale = 0.30
    no_improve = 0

    for j in range(1, N_LOCAL + 1):
        run_id += 1
        cand = sample_local(best_params, base_scale=base_scale)
        rmse_reg, sse, n_tot, n_ok = objective(cand, header, disease_name, control_df, observed_df, run_id=run_id)

        if np.isfinite(rmse_reg) and rmse_reg < best_rmse:
            best_rmse   = rmse_reg
            best_params = cand.copy()
            best_meta   = (sse, n_tot, n_ok)
            no_improve  = 0
            print(f"  [B {j:03d}/{N_LOCAL}] >>> Novo melhor RMSE = {best_rmse:.4f}  (pontos: {n_tot}, exps ok: {n_ok}/{len(control_df)})")
        else:
            no_improve += 1
            # encolhe a trust region se ficar muito tempo sem melhorar
            if no_improve % PATIENCE == 0:
                base_scale *= SHRINK
                base_scale = max(base_scale, 0.05)  # não deixa colapsar
                print(f"    (sem melhora por {PATIENCE} iterações) shrink -> base_scale={base_scale:.3f}")

    # ===== Resultado final =====
    sse, n_tot, n_ok = best_meta
    print(f"\n--- Calibração Biológica para a Região '{REGION_NAME}' Finalizada ---")
    print(f"Melhor RMSE regional: {best_rmse:.4f}  (pontos: {n_tot}, exps ok: {n_ok}/{len(control_df)})")

    # Salva parâmetros (sempre no Desktop/parametros)
    best_params_out = best_params.copy()
    best_params_out['RMSE_regional'] = best_rmse
    saved_path = save_params_safely(best_params_out, REGION_NAME, OUTPUT_DIR)

    # ===== Validação e gráfico =====
    print("\nGerando gráfico de validação com os melhores parâmetros...")
    # escreve os melhores params e re-rodamos para plotar
    write_params_to_file(best_params, header, disease_name, sim_number=99999)

    final_results = {}
    for _, ex in control_df.iterrows():
        exp_file_name = f"{ex['Experiment']}.SBX"
        curve_id = ex['Id_curva']
        if run_dssat_simulation(exp_file_name) and os.path.exists(DSSAT_OUTPUT_FILE):
            try:
                df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)

                obs_curve = observed_df[observed_df['Curva'] == curve_id]
                if obs_curve.empty:
                    last_obs_day = None
                else:
                    last_obs_day = int(pd.to_numeric(obs_curve['Av_DAS'], errors='coerce').max())

                sim_df = calculate_severity(df_out, last_obs_day=last_obs_day)

                rmse, r2, merged_df, sim_df_filtrado, _, _ = evaluate_simulation(sim_df, observed_df, curve_id)
                if merged_df is not None:
                    final_results[curve_id] = {
                        'sim_df': sim_df_filtrado,
                        'merged_df': merged_df,
                        'rmse': rmse,
                        'r2': r2
                    }
            except Exception:
                continue

    if final_results:
        plot_regional_results(final_results, control_df, REGION_NAME, OUTPUT_DIR)
    else:
        print("Sem resultados válidos para plotagem final.")

if __name__ == "__main__":
    main()

Fase A (global): 220 amostras enviesadas por sensibilidade...
  [A 001/220] Melhor RMSE regional = 23.3299  (pontos: 49, exps ok: 8/8)


KeyboardInterrupt: 